<a href="https://colab.research.google.com/github/DenisR144/healthdata22/blob/main/Dataton_2022_(DR_and_T_SH).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Основные действия по основной информации**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Подключение основных библиотек**

In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import sqlite3 as db

**Создание основной БД (без текстовых полей)**

In [ ]:
df = pd.read_csv("drive/MyDrive/DataMed.csv")
#print(df)

conn = db.connect(':memory:')
cur = conn.cursor()
df.to_sql('table1', conn, if_exists='replace', index=False)
#UID;Date;Name;DOB;MKB_1;MKB_2;MKB_3;Died;Violations;ViolationsCausedDeath;Hospital;Ambulance;Home;Meddoc
res = cur.execute("SELECT * FROM table1")
row = res.fetchall()

row

[('1;2022-03-15;GGT;2021-07-21;C;61;2;1;0;0;1;0;1;1.txt',),
 ('2;2022-02-21;ATM;2018-07-20;О;26;8;1;0;0;1;1;0;2.txt',),
 ('3;2021-12-01;CAM;1929-07-08;I;63;0;1;1;0;1;0;0;3.txt',)]

**Подгрузка текстов**

**Обработка текстов**

1) Стемминг (отрезаем в словах окончания и суффиксы), есть вроде готовые библиотеки под русский язык //минусы - теряем часть информации, а часть - сливается

*или* 

2) Лемматизация (морфологический анализ) с целью логического объединения разных форм (например "шёл", "идти" - перевод в слов в начальные формы) - библиотеки pymorphy2, pymystem3

3) частотный анализ, т.е. закидываем все слова с частотным анализом в матрицу, и дальше обрабатываем как вектора
Это надо делать после выше приведенных 1 или 2

4) составление матрицы по частоте использования частей речи (глагол, существительное, прилагательное), перевод их в процентное соотношение от объема текста

1. избавимся от всех лишних символов, кроме русских букв, дефисов и пробелов.


2. цепочка дат событий: приведение дат к единому формату, выстраивание цепочек

3. создание словаря - значимых терминов (от Дениса Рощина, максимальная вероятность получить нормальный словарь)
далее слова из этого словаря найти в текстах и по каждой истории болезни вывести частотную характеристику по каждому слову словаря
В словаре могут быть синонимы.
Статистику по синонимам объединить.

Со словарем значимых слов мне кажется будет решение изящнее...

**Основной анализ обработанного текста и основных полей**

**Всяческая визуализация**